In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle

In [2]:
df = pd.read_csv('./dataset/Restaurant_Reviews.tsv',sep='\t')
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [3]:
X = df['Review']
X

0                               Wow... Loved this place.
1                                     Crust is not good.
2              Not tasty and the texture was just nasty.
3      Stopped by during the late May bank holiday of...
4      The selection on the menu was great and so wer...
                             ...                        
995    I think food should have flavor and texture an...
996                             Appetite instantly gone.
997    Overall I was not impressed and would not go b...
998    The whole experience was underwhelming, and I ...
999    Then, as if I hadn't wasted enough of my life ...
Name: Review, Length: 1000, dtype: object

In [13]:
tokenizer = Tokenizer(10000, lower=True, oov_token='UNK')
tokenizer.fit_on_texts(X)

words = tokenizer.texts_to_sequences(X)

In [25]:
tokenizer.word_counts

OrderedDict([('wow', 3),
             ('loved', 10),
             ('this', 143),
             ('place', 106),
             ('crust', 2),
             ('is', 171),
             ('not', 116),
             ('good', 95),
             ('tasty', 11),
             ('and', 393),
             ('the', 585),
             ('texture', 3),
             ('was', 295),
             ('just', 35),
             ('nasty', 3),
             ('stopped', 2),
             ('by', 21),
             ('during', 5),
             ('late', 2),
             ('may', 5),
             ('bank', 1),
             ('holiday', 1),
             ('off', 9),
             ('rick', 1),
             ('steve', 1),
             ('recommendation', 3),
             ('it', 134),
             ('selection', 10),
             ('on', 55),
             ('menu', 15),
             ('great', 70),
             ('so', 66),
             ('were', 63),
             ('prices', 10),
             ('now', 12),
             ('i', 305),
             ('am',

In [21]:
with open('./tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
with open('./tokenizer.pickle','rb') as myhandle:
    tokenizer1 = pickle.load(myhandle)

In [5]:
tokenizer1.word_counts

OrderedDict([('wow', 3),
             ('loved', 10),
             ('this', 143),
             ('place', 106),
             ('crust', 2),
             ('is', 171),
             ('not', 116),
             ('good', 95),
             ('tasty', 11),
             ('and', 393),
             ('the', 585),
             ('texture', 3),
             ('was', 295),
             ('just', 35),
             ('nasty', 3),
             ('stopped', 2),
             ('by', 21),
             ('during', 5),
             ('late', 2),
             ('may', 5),
             ('bank', 1),
             ('holiday', 1),
             ('off', 9),
             ('rick', 1),
             ('steve', 1),
             ('recommendation', 3),
             ('it', 134),
             ('selection', 10),
             ('on', 55),
             ('menu', 15),
             ('great', 70),
             ('so', 66),
             ('were', 63),
             ('prices', 10),
             ('now', 12),
             ('i', 305),
             ('am',

In [6]:
with open('./model/tokenizer.pickle','rb') as tok_handle:
    new_tokenizer = pickle.load(tok_handle)

In [7]:
from tensorflow.keras.models import load_model

In [8]:
model = load_model('./model/restaurant.h5')

In [9]:
def get_review(review):
    sentence_list = []
    sentence_list.append(review)
    sequences = new_tokenizer.texts_to_sequences(sentence_list)
    input = pad_sequences(sequences, maxlen=200, padding='pre')
    output = model.predict(input)
    print(output[0])
    
    if output[0] >= 0.5:    
        print("The feedback from user is Positive")    
        return 1
    else:
        print("The feedback from user is Negative")
        return 0 

In [10]:
get_review("the food was not good")

1/1 [==============================] - 9s 9s/step
[0.06175401]
The feedback from user is Negative


0

In [11]:
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
import json

In [12]:
app = Flask(__name__)
CORS(app)

@app.route('/',methods=['GET'])
def Home():
    return render_template('index.html')

@app.route('/review', methods = ['POST'])
def review():
    if request.method == 'POST':
        record = request.get_json()    
        user_feedback = record['review']
        prediction_val = get_review(user_feedback)    
        prediction = {}
        prediction['sentiment'] = prediction_val
        prediction['review'] = record['review']
        print(prediction)        
        return json.dumps(prediction)
    
if __name__ == "__main__":
    app.run(use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Dec/2022 17:27:15] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 148ms/step


127.0.0.1 - - [01/Dec/2022 17:27:24] "POST /review HTTP/1.1" 200 -


[0.02755329]
The feedback from user is Negative
{'sentiment': 0, 'review': 'The food was worst'}
1/1 [==============================] - 0s 148ms/step


...

[0.9990959]
The feedback from user is Positive
{'sentiment': 1, 'review': 'The food was great'}
1/1 [==============================] - 0s 162ms/step


127.0.0.1 - - [01/Dec/2022 17:27:53] "POST /review HTTP/1.1" 200 -


[0.04477699]
The feedback from user is Negative
{'sentiment': 0, 'review': 'I did not like the sweet dish'}
1/1 [==============================] - 0s 165ms/step


127.0.0.1 - - [01/Dec/2022 17:28:10] "POST /review HTTP/1.1" 200 -


[0.9587721]
The feedback from user is Positive
{'sentiment': 1, 'review': 'I liked sushi more'}


In [19]:
print(pd.__version__)
print(np.__version__)
print(tf.__version__)

#print(pickle) #0.7.5
# Flask 2.1.3
#flask-restful 0.3.8
#flask_cors 3.0.10
#matplotlib                3.5.2
#seaborn                   0.12.0 
#scikit-learn              1.1.2

1.4.4
1.21.5
2.10.0


NameError: name 'flask' is not defined